Generate dataset

In [1]:
%matplotlib inline
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
from IPython import display

import sys
sys.path.append("..")
import d2dl

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs, dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
epsilon = torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)
labels += epsilon

Load datas

In [3]:
import torch.utils.data as Data

datasets = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(datasets, batch_size=10, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.3188,  0.9716],
        [-0.3590,  1.4006],
        [-0.1155, -1.0571],
        [ 0.3579,  1.2973],
        [-0.0601,  1.3941],
        [ 0.6582,  0.5168],
        [-0.3004,  0.7757],
        [ 1.5631,  1.3011],
        [ 1.0798,  0.2168],
        [-0.5072, -0.7901]]) tensor([ 1.5474, -1.2791,  7.5662,  0.5243, -0.6536,  3.7538,  0.9709,  2.9174,
         5.6230,  5.8609])


Define model

In [4]:
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_features):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_features, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

for param in net.parameters():
    print(param)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[0.0839, 0.1168]], requires_grad=True)
Parameter containing:
tensor([0.0592], requires_grad=True)


Initialize

In [5]:
from torch.nn import init

init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

Loss function

In [6]:
loss = nn.MSELoss()

Optimizer

In [7]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


Train

In [8]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch + 1, l.item()))

epoch 1, loss: 0.000340
epoch 2, loss: 0.000058
epoch 3, loss: 0.000035


Compare

In [9]:
dense = net.linear
print(true_w, "\n", dense.weight)
print(true_b, "\n", dense.bias)

[2, -3.4] 
 Parameter containing:
tensor([[ 1.9992, -3.4000]], requires_grad=True)
4.2 
 Parameter containing:
tensor([4.1997], requires_grad=True)
